In [2]:
import pandas as pd 
import numpy as np 
import requests
import json
import dotenv
import os

import sys
sys.tracebacklimit = 0

In [3]:
dotenv.load_dotenv()
spotify_client_id = os.getenv('spotify_clientid')
spotify_client_secret = os.getenv('spotify_clientsecret')


url = 'https://accounts.spotify.com/api/token'
headers = {'Content-Type': 'application/x-www-form-urlencoded',}
data = {
    'grant_type': 'client_credentials',
    'client_id': spotify_client_id,
    'client_secret': spotify_client_secret,
}

response = requests.post(url, headers=headers, data=data)

token_info = response.json()
access_token = token_info['access_token']

In [10]:
artist_id = "4YRxDV8wJFPHPTeXepOstw"
options = "appears_on"
limit = "50"
url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups={options}&limit={limit}'
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get(url, headers=headers) 
total_album_no = json.loads(response.content)['total']

album_dict = {}
print(np.arange(0,total_album_no,50))
for i in np.arange(0,total_album_no,50):
    print(i)
    offset = i
    url = f'https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups={options}&limit={limit}&offset={offset}'
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers) 

    df = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)
    album_dict[i] = df
album_df = pd.concat(album_dict.values())


[   0   50  100  150  200  250  300  350  400  450  500  550  600  650
  700  750  800  850  900  950 1000 1050 1100 1150 1200]
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200


In [11]:
album_df['Album_ID'] = album_df['external_urls.spotify'].apply(lambda x: x.split("/")[-1])
album_df['Producer'] = album_df['artists'].apply(lambda x: x[0]['name'])
album_df['Producer_ID'] = album_df['artists'].apply(lambda x: x[0]['id'])
album_df['name'] = album_df['name'].apply(lambda x: x.replace("(Original Motion Picture Soundtrack)",""))
album_df.to_csv("albums.csv")
album_df


,album_type,total_tracks,href,id,images,name,release_date,release_date_precision,type,uri,artists,album_group,external_urls.spotify,Album_ID,Producer,Producer_ID
0,album,12,https://api.spotify.com/v1/albums/0Rkv5iqjF2ue...,0Rkv5iqjF2uenfL0OVB8hg,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Aashiqui 2,2013-04-06,day,album,spotify:album:0Rkv5iqjF2uenfL0OVB8hg,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0Rkv5iqjF2uenfL...,0Rkv5iqjF2uenfL0OVB8hg,Mithoon,09UmIX92EUH9hAK4bxvHx6
1,album,9,https://api.spotify.com/v1/albums/2Lxoc72vRTGd...,2Lxoc72vRTGdQfMvj7Ovi1,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Yeh Jawaani Hai Deewani,2013-03-30,day,album,spotify:album:2Lxoc72vRTGdQfMvj7Ovi1,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/2Lxoc72vRTGdQfM...,2Lxoc72vRTGdQfMvj7Ovi1,Pritam,1wRPtKGflJrBx9BmLsSwlU
2,album,8,https://api.spotify.com/v1/albums/0a183xiCHiC1...,0a183xiCHiC1GQd8ou7WXO,[{'url': 'https://i.scdn.co/image/ab67616d0000...,ANIMAL,2023-11-24,day,album,spotify:album:0a183xiCHiC1GQd8ou7WXO,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0a183xiCHiC1GQd...,0a183xiCHiC1GQd8ou7WXO,Manan Bhardwaj,3pQ4aA7dkolyjUAMrVScgh
3,album,16,https://api.spotify.com/v1/albums/0LcyzKKw3RjF...,0LcyzKKw3RjFKL6ygISTeU,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Ek Tha Raja,2024-03-18,day,album,spotify:album:0LcyzKKw3RjFKL6ygISTeU,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0LcyzKKw3RjFKL6...,0LcyzKKw3RjFKL6ygISTeU,Badshah,0y59o4v8uw5crbN9M3JiL1
4,album,6,https://api.spotify.com/v1/albums/7sVzilmsqYcL...,7sVzilmsqYcLMYw2gtvoZM,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Hamari Adhuri Kahani,2015-05-21,day,album,spotify:album:7sVzilmsqYcLMYw2gtvoZM,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/7sVzilmsqYcLMYw...,7sVzilmsqYcLMYw2gtvoZM,Jeet Gannguli,2kkQthS9OLpK4UqNWYqoVl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,compilation,10,https://api.spotify.com/v1/albums/0vLCAjawZPbM...,0vLCAjawZPbMtyhR6TcGED,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Dastak - A Bouquet Of Ghazals,2015-01-01,day,album,spotify:album:0vLCAjawZPbMtyhR6TcGED,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/0vLCAjawZPbMtyh...,0vLCAjawZPbMtyhR6TcGED,Various Artists,0LyfQWJT6nXafLPZqxe9Of
35,compilation,7,https://api.spotify.com/v1/albums/35MkN8qsxx5K...,35MkN8qsxx5KEELxxISsak,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Celebrating World Music Day With Indipop Chart...,2015-06-18,day,album,spotify:album:35MkN8qsxx5KEELxxISsak,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/35MkN8qsxx5KEEL...,35MkN8qsxx5KEELxxISsak,Various Artists,0LyfQWJT6nXafLPZqxe9Of
36,compilation,1,https://api.spotify.com/v1/albums/2obuaVKDXzbu...,2obuaVKDXzbuTNU9Tx0HfK,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Phir Aur Kya Chahiye (Urban Drill Mix),2023-08-11,day,album,spotify:album:2obuaVKDXzbuTNU9Tx0HfK,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/2obuaVKDXzbuTNU...,2obuaVKDXzbuTNU9Tx0HfK,Various Artists,0LyfQWJT6nXafLPZqxe9Of
37,compilation,13,https://api.spotify.com/v1/albums/1GKO9IYiDGCC...,1GKO9IYiDGCCcwWNNV0JGV,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Dance To The Tunes Turn This Party Up Vol-1,2020-09-23,day,album,spotify:album:1GKO9IYiDGCCcwWNNV0JGV,[{'external_urls': {'spotify': 'https://open.s...,appears_on,https://open.spotify.com/album/1GKO9IYiDGCCcwW...,1GKO9IYiDGCCcwWNNV0JGV,Various Artists,0LyfQWJT6nXafLPZqxe9Of


In [12]:
all_albums = {}
for i,album_id in enumerate(album_df['Album_ID'].unique()):
    print(i,len(album_df))
    url = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
    response = requests.get(url, headers=headers) 
    
    df_t = pd.json_normalize(json.loads(response.text), record_path=['items']).drop("available_markets",axis=1)
    df_t['Album_ID'] = album_id

    df_t['Artist Song'] = df_t['artists'].apply(lambda x: artist_id in str(x))
    all_albums[album_id] = df_t

df_tracks = pd.concat(all_albums.values())
df_tracks.to_csv("tracks.csv")

0 1239
1 1239
2 1239
3 1239
4 1239
5 1239
6 1239
7 1239
8 1239
9 1239
10 1239
11 1239
12 1239
13 1239
14 1239
15 1239
16 1239
17 1239
18 1239
19 1239
20 1239
21 1239
22 1239
23 1239
24 1239
25 1239
26 1239
27 1239
28 1239
29 1239
30 1239
31 1239
32 1239
33 1239
34 1239
35 1239
36 1239
37 1239
38 1239
39 1239
40 1239
41 1239
42 1239
43 1239
44 1239
45 1239
46 1239
47 1239
48 1239
49 1239
50 1239
51 1239
52 1239
53 1239
54 1239
55 1239
56 1239
57 1239
58 1239
59 1239
60 1239
61 1239
62 1239
63 1239
64 1239
65 1239
66 1239
67 1239
68 1239
69 1239
70 1239
71 1239
72 1239
73 1239
74 1239
75 1239
76 1239
77 1239
78 1239
79 1239
80 1239
81 1239
82 1239
83 1239
84 1239
85 1239
86 1239
87 1239
88 1239
89 1239
90 1239
91 1239
92 1239
93 1239
94 1239
95 1239
96 1239
97 1239
98 1239
99 1239
100 1239
101 1239
102 1239
103 1239
104 1239
105 1239
106 1239
107 1239
108 1239
109 1239
110 1239
111 1239
112 1239
113 1239
114 1239
115 1239
116 1239
117 1239
118 1239
119 1239
120 1239
121 1239
122 1239
123

In [14]:
artist_songs = df_tracks[df_tracks['Artist Song']==True]
artist_songs = artist_songs[['artists','duration_ms','id','name','Album_ID']].rename(columns = {'name':"Track_Name",
                                                                                                'artists':"Track_Artists"})

album_details = album_df[['total_tracks','name','release_date','artists','Album_ID','Producer','Producer_ID']].rename(columns = {'name':'Album_Name',
                                                                                                                                'artists':"Album_Artists"})

final = artist_songs.merge(album_details, on = "Album_ID", how = "left", indicator=True)
print(final._merge.value_counts())
final.to_csv("All_Details_Full.csv", index = False)
final

_merge
both          2051
left_only        0
right_only       0
Name: count, dtype: int64


,Track_Artists,duration_ms,id,Track_Name,Album_ID,total_tracks,Album_Name,release_date,Album_Artists,Producer,Producer_ID,_merge
0,[{'external_urls': {'spotify': 'https://open.s...,261974,56zZ48jdyY2oDXHVnwg5Di,Tum Hi Ho,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
1,[{'external_urls': {'spotify': 'https://open.s...,304899,6UgcN95w7vQxkR8sEFmwHG,Chahun Main Ya Naa,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
2,[{'external_urls': {'spotify': 'https://open.s...,306511,55eUtE9v3uatR0y0NijM6N,Hum Mar Jayenge,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
3,[{'external_urls': {'spotify': 'https://open.s...,266069,3E9ryvrKP3hUWX79PAakH2,Meri Aashiqui,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
4,[{'external_urls': {'spotify': 'https://open.s...,214470,06zJUeKU5KFy50Cho6pDcr,Aasan Nahin Yahan,0Rkv5iqjF2uenfL0OVB8hg,12,Aashiqui 2,2013-04-06,[{'external_urls': {'spotify': 'https://open.s...,Mithoon,09UmIX92EUH9hAK4bxvHx6,both
...,...,...,...,...,...,...,...,...,...,...,...,...
2046,[{'external_urls': {'spotify': 'https://open.s...,317413,0zciOy8kgToHnOc7m3RrAQ,Rishton Ke Saare Manzar,0vLCAjawZPbMtyhR6TcGED,10,Dastak - A Bouquet Of Ghazals,2015-01-01,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both
2047,[{'external_urls': {'spotify': 'https://open.s...,317348,21Nv3TlVXps943IGFWuJ8S,Rishton Ke Saare Manzar,35MkN8qsxx5KEELxxISsak,7,Celebrating World Music Day With Indipop Chart...,2015-06-18,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both
2048,[{'external_urls': {'spotify': 'https://open.s...,50749,3zk5QbltmuZWH8VapbdP4N,Phir Aur Kya Chahiye - Urban Drill Mix,2obuaVKDXzbuTNU9Tx0HfK,1,Phir Aur Kya Chahiye (Urban Drill Mix),2023-08-11,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both
2049,[{'external_urls': {'spotify': 'https://open.s...,200247,32t0LPcifv1tX4sGwrhBGE,"Hayi Hayi (From ""Dohchay"")",1GKO9IYiDGCCcwWNNV0JGV,13,Dance To The Tunes Turn This Party Up Vol-1,2020-09-23,[{'external_urls': {'spotify': 'https://open.s...,Various Artists,0LyfQWJT6nXafLPZqxe9Of,both


In [15]:
final_graph = final.copy()
final_graph = final_graph[final_graph['Producer']!="Various Artists"]
final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Best" not in x.title())]
final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Top" not in x.title())]
final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Hits" not in x.title())]

final_graph = final_graph[final_graph['Album_Name'].apply(lambda x: "Remix" not in x.title())]
final_graph = final_graph[final_graph['Track_Name'].apply(lambda x: "Remix" not in x.title())]
final_graph = final_graph[final_graph['Track_Name'].apply(lambda x: "Mix" not in x.title())]
print(final_graph)
final_graph.to_csv("All_Details.csv", index = False)


                                          Track_Artists  duration_ms  \
0     [{'external_urls': {'spotify': 'https://open.s...       261974   
1     [{'external_urls': {'spotify': 'https://open.s...       304899   
2     [{'external_urls': {'spotify': 'https://open.s...       306511   
3     [{'external_urls': {'spotify': 'https://open.s...       266069   
4     [{'external_urls': {'spotify': 'https://open.s...       214470   
...                                                 ...          ...   
1930  [{'external_urls': {'spotify': 'https://open.s...       188813   
1934  [{'external_urls': {'spotify': 'https://open.s...       219193   
1974  [{'external_urls': {'spotify': 'https://open.s...       192400   
2011  [{'external_urls': {'spotify': 'https://open.s...       210619   
2043  [{'external_urls': {'spotify': 'https://open.s...       270624   

                          id                            Track_Name  \
0     56zZ48jdyY2oDXHVnwg5Di                             Tum Hi H